## 第2版本

### Web多应用综合，初步完成demo

#### 目的，demo，调试日志，多层次测试，手动构建与生产部署
  


  - 业务核心功能
    - 核心业务梳理，模块划分
    - 数据库设计
    - MySQL,MongoDB,Redis异步操作
    - 数据的CUDR服务
    - 路由接口设计
    - 认证授权
    - 异常处理
    - 安全防护
      - 防SQL注入
      - XSS
      - CSFR
    - 性能优化
      - 缓存
      - 异步任务
      - 负载均衡
      - 多进程多线程
  - 调试和日志信息
  - 单元测试，集成测试，端点测试，负载测试
  - 简单手动镜像构建和生产部署

核心功能点:
1. 用户注册只能通过邀请码
2. 用户主账号可以关联两个匿名账号
3. 朋友关系可达6度（朋友的朋友的朋友...最多6层），并且可以查看关系链
4. 用户之间可以设置熟悉等级（陌生，了解，熟悉，信任）
5. 用户可以关注其他用户
6. 用户可以写博客（支持标签、草稿、搜索、评论回复），并且评论回复可以转化为帖子
7. 用户可以对博客标记喜欢、不喜欢、收藏
8. 用户之间可以建立私聊或群聊，聊天内容可以转化为帖子
9. 用户可以直接创建帖子(支持标签，用户回复),可对帖子标记喜欢，不喜欢，收藏

#### 用户应用

- 根据邀请码，注册用户
- 用户主账号可以关联两个匿名账号。
- 朋友关系可达6度（朋友的朋友的朋友...最多6层），并且可以查看关系链。
- 用户之间可以设置熟悉等级（陌生，了解，熟悉，信任）。
- 用户可以关注其他用户。

MySQL数据库设计（用于存储结构化数据，如用户信息、关系、邀请码等）
- 用户表users
    - id: 主键
    - serial: 用户序列（唯一）
    - username: 用户名（非唯一）
    - type: anonymous|primary
    - email: 邮箱（非唯一）
    - phone：手机（非唯一）
    - password_hash: 密码哈希
    - salt: 椒盐值
    - created_at: 创建时间
    - last_login: 最后登录时间
    - status: 状态（在线、禁用、隐身）
- 用户权限表user_privileges
    - id
    - user_id
    - invitation_code_num
    - create_invitation
    - anonymous_num
    - create_anonymous
    - relation_gap
- 主账号与匿名账号关联表user_anonymous_accounts
    - id: 主键
    - primary_user_id: 主账号ID（外键到users.id）
    - anonymous_user_id: 匿名账号ID（外键到users.id）
    - created_at: 创建时间
- 邀请码表invitation_codes
    - code: 邀请码（主键）
    - created_by: 创建者（可能是管理员或用户，如果允许用户生成邀请码，则记录；否则可以是系统）
    - created_at: 创建时间
    - used_by: 被哪个用户使用（外键到users.id，可为空）
    - used_at: 使用时间
- 用户关系表user_relationships，仅记录gap=0的关系
    - id: 主键
    - user_id: 用户ID（外键到users.id）
    - related_user_id: 相关用户ID（外键到users.id）
    - level: 熟悉等级（0:陌生, 1:了解, 2:熟悉, 3:信任）
    - created_at: 关系创建时间
    - updated_at: 最后更新时间
- 关注表follows
    - follower_id: 关注者ID
    - followed_id: 被关注者ID
    - created_at
    - 主键 (follower_id, followed_id)

注意事项：
- 匿名账号也有朋友圈等功能，完全和主账号一样
- 匿名账号和主账号可以互相访问对方的资源
- 主账号为了信任机制，必须填写正确内容，匿名帐号没限制
- 匿名账号可以销毁和重建，前期可以设定数量
- 每个人的邀请码也可以限制数量
- 邀请注册，邀请码和serial别人看不见
- 朋友的定义，认为是双向都同意(可以实现沟通)，才能创建，并必须附带关系等级
- 邀请码默认朋友，朋友相互点亮(和相互关注解耦)，关系等级
- gap=设置值的关系链，在redis中查看，避免重复计算
- 用户还有职业，标签，项目，技能，经历等信息，方便日后做搭配使用，现在先不设计，既可以mysql设计多个单表，也可以用mongodb存储半结构数据，方便扩展

#### 博客应用

- 用户可以写博客，支持标签，草稿，搜索，评论回复
- 评论回复可以转换成帖子
- 用户可以对博客标记喜欢，不喜欢，收藏

#### 聊天应用

- 用户之间可以建立私聊或群聊
- 聊天内容可以转换成帖子

#### 论坛应用

- 用户可以直接创建帖子，支持标签，用户回复
- 可对帖子标记喜欢，不喜欢，收藏